In [18]:
from supercats import add_supercats
import import_data
import pandas as pd
from simplekml import Kml, Style
from colorsys import hsv_to_rgb as color

#Little helper function to restrict to one city because google earth is helpless otherwise
def get_by_city(city):
    return businesses[businesses['city']==city].copy(deep=True)

#second helper for color conv, copies from activestate.com and slightly modfied
def format_color(rgb_tuple):
    """ convert an (R, G, B) tuple to #RRGGBBff """
    rgb_tuple = (rgb_tuple[0]*255, rgb_tuple[1]*255, rgb_tuple[2]*255)
    hexcolor = '%02x%02x%02x' % rgb_tuple
    return hexcolor

In [10]:
# Import businesses
businesses = import_data.import_businesses()

In [11]:
# Add supercats
(df, supercats) = add_supercats(businesses)

In [35]:
def kml_by_city(city):
    #Find supercats
    (df, supercats) = add_supercats(get_by_city('Edinburgh'))
    
    # Prepare KML
    kml = Kml()

    #Create folder for businesses
    fol = kml.newfolder(name="Businesses")

    #Create subfolder for every category and save them in dictionary
    subfolders = {key: fol.newfolder(name=supercats[key]) for key in supercats}

    #Create color for every category and also save them in dict
    styles = {key: Style() for key in supercats}
    for key in supercats:
        styles[key].labelstyle.scale = 0
        styles[key].iconstyle.icon.href = 'http://thydzik.com/thydzikGoogleMap/markerlink.php?color=%s' % format_color(color(key/len(supercats), 1, 1))

    #Special case for uncategories (those are -1 and will be black)
    styles[-1] = Style()
    styles[-1].labelstyle.scale = 0
    styles[-1].iconstyle.icon.href = 'http://thydzik.com/thydzikGoogleMap/markerlink.php?color=000000'
    
    for item in df[['name', 'longitude', 'latitude','super category']].values:
        pnt = subfolders[item[3]].newpoint(name=item[0], coords=[(item[1],item[2])])
        pnt.style = styles[item[3]]

    kml.save("%s.kml" % city)

In [13]:
for item in get_by_city('Las Vegas')[['name', 'longitude', 'latitude','super category']].values:
    pnt = subfolders[item[3]].newpoint(name=item[0], coords=[(item[1],item[2])])
    pnt.style = styles[item[3]]

kml.save("busi.kml")

In [28]:
(df, supercats) = add_supercats(get_by_city('Edinburgh'))

In [27]:
for item in df[['name', 'longitude', 'latitude','super category']].values:
    pnt = subfolders[item[3]].newpoint(name=item[0], coords=[(item[1],item[2])])
    pnt.style = styles[item[3]]

kml.save("lasvegas.kml")

In [36]:
kml_by_city('Edinburgh')